# Imports

In [6]:
import numpy as np
import scipy.spatial.distance
from ui_script import show_top_5
import pickle
import random
import re

# Load word2vec

In [2]:
from gensim.models.word2vec import Word2Vec
w2v = Word2Vec.load_word2vec_format('../../../data/GoogleNews-vectors-negative300.bin', binary=True)
w2v.init_sims(replace=True)

# Load image vectors

In [3]:
new_vec_dict = np.load('image_space.npy').item()

# Evaluate a query image

In [31]:
# Random select query image and caption
query_file_name = random.choice(list(new_vec_dict.keys()))
query_vec = new_vec_dict[query_file_name]
query_caption = w2v.most_similar(query_vec, restrict_vocab=30000, topn=1)[0][0]

# top 5 images
top5_file_names = ['a', 'a', 'a', 'a', 'a']
top5_captions = ['a', 'a', 'a', 'a', 'a']
top5_distances = [1.0, 1.0, 1.0, 1.0, 1.0]

# Iterate over all images
for file_name in new_vec_dict.keys():
    if file_name != query_file_name:
        
#        # Iterate over all captions belonging to an image
#        closest_distance = 1.0
#        idx = -1
#        for i, w2v in enumerate(w2v_dic[file_name]):
#            dis = scipy.spatial.distance.cosine(query_w2v,w2v)
#            if dis < closest_distance:
#                closest_distance = dis
#                idx = i

        # See if closest caption is in top 5
        for i, dist in enumerate(top5_distances):
            temp_dist = scipy.spatial.distance.cosine(query_vec,new_vec_dict[file_name])
            if temp_dist < dist:
                top5_distances[i] = temp_dist
#                top5_captions[i] = caption_dic[file_name][idx] + "; " + str(round(closest_distance, 4))
                top5_file_names[i] = '../../../data/train2014/' + file_name
#                top5_idx[i] = idx
                break

# Find most similar word in word2vec
for i in range(5):
    file_name = re.search('COCO.+',top5_file_names[i])[0]
    top5_captions[i] =w2v.most_similar(
        np.mean([query_vec,new_vec_dict[file_name]],axis=0), restrict_vocab=30000, topn=1)[0][0]

# Display result
file_names = ['../../../data/train2014/' + query_file_name] + top5_file_names
captions = [query_caption] + top5_captions
show_top_5(file_names, captions, height = 250, title='Image space evaluation')